> **Tip**: Welcome to the Investigate a Dataset project! You will find tips in quoted sections like this to help organize your approach to your investigation. Once you complete this project, remove these **Tip** sections from your report before submission. First things first, you might want to double-click this Markdown cell and change the title so that it reflects your dataset and investigation.

# Project: Investigate a Dataset - [Dataset-name]

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

### Dataset Description 

> **Tip**: In this section of the report, provide a brief introduction to the dataset you've selected/downloaded for analysis. Read through the description available on the homepage-links present [here](https://docs.google.com/document/d/e/2PACX-1vTlVmknRRnfy_4eTrjw5hYGaiQim5ctr9naaRd4V9du2B5bxpd8FEH3KtDgp8qVekw7Cj1GLk1IXdZi/pub?embedded=True). List all column names in each table, and their significance. In case of multiple tables, describe the relationship between tables. 


### Question(s) for Analysis
>**Tip**: Clearly state one or more questions that you plan on exploring over the course of the report. You will address these questions in the **data analysis** and **conclusion** sections. Try to build your report around the analysis of at least one dependent variable and three independent variables. If you're not sure what questions to ask, then make sure you familiarize yourself with the dataset, its variables and the dataset context for ideas of what to explore.

> **Tip**: Once you start coding, use NumPy arrays, Pandas Series, and DataFrames where appropriate rather than Python lists and dictionaries. Also, **use good coding practices**, such as, define and use functions to avoid repetitive code. Use appropriate comments within the code cells, explanation in the mark-down cells, and meaningful variable names. 

In [1]:
# Libraries and modules in alphabetical order
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sqlite3

In [2]:
conn = sqlite3.connect("../database.sqlite")

In [3]:
query = '''
            WITH home_team as (
SELECT  match_api_id, team_api_id, team_long_name, team_short_name	
FROM  "Match" 
JOIN Team ON Match.home_team_api_id  = team.team_api_id
),
away_team as (
SELECT  match_api_id, team_api_id, team_long_name, team_short_name	
FROM  "Match" 
JOIN Team ON Match.away_team_api_id  = team.team_api_id
),

htm_attribute as(
SELECT match_api_id, team_api_id, ta.date as hdate,
	buildUpPlaySpeed as ht_bup_speed, buildUpPlayDribbling as ht_bup_dribbling, buildUpPlayPassing as ht_bup_passing,
	chanceCreationPassing as ht_cc_passing, chanceCreationCrossing as ht_cc_crossing, chanceCreationShooting as ht_cc_shooting, 
	defencePressure as ht_df_pressure, defenceAggression as ht_df_aggression, defenceTeamWidth as ht_df_teamwidth
FROM   Team_Attributes as ta
JOIN "Match" ON Match.home_team_api_id  = ta.team_api_id
WHERE ta.date LIKE "%2015%"
),
atm_attribute as(
SELECT match_api_id, team_api_id, ta.date as adate,
	buildUpPlaySpeed as at_bup_speed, buildUpPlayDribbling as at_bup_dribbling, buildUpPlayPassing as at_bup_passing,
	chanceCreationPassing as at_cc_passing, chanceCreationCrossing as at_cc_crossing, chanceCreationShooting as at_cc_shooting, 
	defencePressure as at_df_pressure, defenceAggression as at_df_aggression, defenceTeamWidth as at_df_teamwidth
FROM   Team_Attributes as ta
JOIN "Match" ON Match.away_team_api_id = ta.team_api_id
WHERE ta.date LIKE "%2015%"
)

SELECT  
	"date", stage
	,home_team_api_id, away_team_api_id
	,htm.team_long_name as home_teams, home_team_goal
	,away_team_goal, atm.team_long_name as away_teams,
	BWH, BWD, BWA, 
	ht_bup_speed, at_bup_speed,  ht_bup_dribbling, at_bup_dribbling, ht_bup_passing, at_bup_passing,
	 ht_cc_passing, at_cc_passing, ht_cc_crossing, at_cc_crossing, ht_cc_shooting, at_cc_shooting, 
	 ht_df_pressure, at_df_pressure, ht_df_aggression,  at_df_aggression, ht_df_teamwidth, at_df_teamwidth
	     
	      
	   
FROM  "Match" 
JOIN home_team as htm  ON Match.match_api_id  = htm.match_api_id  
JOIN away_team as atm ON  Match.match_api_id  = atm.match_api_id
JOIN htm_attribute  as htma ON  match.match_api_id  = htma.match_api_id 
JOIN atm_attribute  as atma ON  match.match_api_id  = atma.match_api_id 


WHERE  
	league_id = 1729 --filter out the English Premier League
	AND (season LIKE "%2015/2016%" OR season LIKE "%2014/2015%")
ORDER BY stage
        '''



In [4]:
df = pd.read_sql(query, conn)
# pd.to_datetime()
df["date"] = pd.to_datetime(df["date"] )
df.head()
df.set_index("date")

,stage,home_team_api_id,away_team_api_id,home_teams,home_team_goal,away_team_goal,away_teams,BWH,BWD,BWA,...,ht_cc_crossing,at_cc_crossing,ht_cc_shooting,at_cc_shooting,ht_df_pressure,at_df_pressure,ht_df_aggression,at_df_aggression,ht_df_teamwidth,at_df_teamwidth
date,,,,,,,,,,,,,,,,,,,,,
2014-08-16,1,9825,9826,Arsenal,2,1,Crystal Palace,1.25,5.50,12.00,...,44,69,46,50,51,36,44,34,52,41
2014-08-18,1,8191,8455,Burnley,1,3,Chelsea,7.50,4.60,1.40,...,57,34,47,44,63,39,62,41,58,46
2014-08-16,1,8197,8668,Leicester City,2,2,Everton,2.90,3.20,2.40,...,64,33,46,63,58,52,65,58,55,59
2014-08-17,1,8650,8466,Liverpool,2,1,Southampton,1.36,4.50,8.75,...,34,54,46,30,51,50,52,65,61,44
2014-08-16,1,10260,10003,Manchester United,1,2,Swansea City,1.40,4.75,9.00,...,44,36,40,55,54,31,53,47,56,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-05-15,38,8466,9826,Southampton,4,1,Crystal Palace,1.33,4.75,9.25,...,54,69,30,50,50,36,65,34,44,41
2016-05-15,38,10194,8654,Stoke City,2,1,West Ham United,3.30,3.40,2.10,...,59,73,49,29,44,30,37,37,51,41
2016-05-15,38,10003,8456,Swansea City,1,1,Manchester City,5.75,4.50,1.48,...,36,36,55,24,31,48,47,47,42,54


In [1]:
# Upgrade pandas to use dataframe.explode() function. 
# !pip install --upgrade pandas==0.25.0

  Running setup.py clean for pandas
Failed to build pandas
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.4
    Uninstalling pandas-1.3.4:
      Successfully uninstalled pandas-1.3.4
    Running setup.py install for pandas: started
    Running setup.py install for pandas: still running...
    Running setup.py install for pandas: finished with status 'error'
  Rolling back uninstall of pandas
  Moving to c:\users\damol\anaconda3\lib\site-packages\pandas-1.3.4.dist-info\
   from C:\Users\damol\anaconda3\Lib\site-packages\~andas-1.3.4.dist-info
  Moving to c:\users\damol\anaconda3\lib\site-packages\pandas\
   from C:\Users\damol\anaconda3\Lib\site-packages\~andas


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\damol\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\damol\\AppData\\Local\\Temp\\pip-install-6x4vlwcw\\pandas_879812575e3948e4af878b9f875370bc\\setup.py'"'"'; __file__='"'"'C:\\Users\\damol\\AppData\\Local\\Temp\\pip-install-6x4vlwcw\\pandas_879812575e3948e4af878b9f875370bc\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\damol\AppData\Local\Temp\pip-wheel-haa9t6pe'
       cwd: C:\Users\damol\AppData\Local\Temp\pip-install-6x4vlwcw\pandas_879812575e3948e4af878b9f875370bc\
  Complete output (907 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.9
  creat

  copying pandas\tests\io\parser\test_dtypes.py -> build\lib.win-amd64-3.9\pandas\tests\io\parser
  copying pandas\tests\io\parser\test_header.py -> build\lib.win-amd64-3.9\pandas\tests\io\parser
  copying pandas\tests\io\parser\test_index_col.py -> build\lib.win-amd64-3.9\pandas\tests\io\parser
  copying pandas\tests\io\parser\test_mangle_dupes.py -> build\lib.win-amd64-3.9\pandas\tests\io\parser
  copying pandas\tests\io\parser\test_multi_thread.py -> build\lib.win-amd64-3.9\pandas\tests\io\parser
  copying pandas\tests\io\parser\test_na_values.py -> build\lib.win-amd64-3.9\pandas\tests\io\parser
  copying pandas\tests\io\parser\test_network.py -> build\lib.win-amd64-3.9\pandas\tests\io\parser
  copying pandas\tests\io\parser\test_parse_dates.py -> build\lib.win-amd64-3.9\pandas\tests\io\parser
  copying pandas\tests\io\parser\test_python_parser_only.py -> build\lib.win-amd64-3.9\pandas\tests\io\parser
  copying pandas\tests\io\parser\test_quoting.py -> build\lib.win-amd64-3.9\pandas

    creating build\lib.win-amd64-3.9\pandas
    copying pandas\conftest.py -> build\lib.win-amd64-3.9\pandas
    copying pandas\testing.py -> build\lib.win-amd64-3.9\pandas
    copying pandas\_typing.py -> build\lib.win-amd64-3.9\pandas
    copying pandas\_version.py -> build\lib.win-amd64-3.9\pandas
    copying pandas\__init__.py -> build\lib.win-amd64-3.9\pandas
    creating build\lib.win-amd64-3.9\pandas\api
    copying pandas\api\__init__.py -> build\lib.win-amd64-3.9\pandas\api
    creating build\lib.win-amd64-3.9\pandas\arrays
    copying pandas\arrays\__init__.py -> build\lib.win-amd64-3.9\pandas\arrays
    creating build\lib.win-amd64-3.9\pandas\compat
    copying pandas\compat\chainmap.py -> build\lib.win-amd64-3.9\pandas\compat
    copying pandas\compat\pickle_compat.py -> build\lib.win-amd64-3.9\pandas\compat
    copying pandas\compat\_optional.py -> build\lib.win-amd64-3.9\pandas\compat
    copying pandas\compat\__init__.py -> build\lib.win-amd64-3.9\pandas\compat
    creat

<a id='wrangling'></a>
## Data Wrangling

> **Tip**: In this section of the report, you will load in the data, check for cleanliness, and then trim and clean your dataset for analysis. Make sure that you **document your data cleaning steps in mark-down cells precisely and justify your cleaning decisions.**


### General Properties
> **Tip**: You should _not_ perform too many operations in each cell. Create cells freely to explore your data. One option that you can take with this project is to do a lot of explorations in an initial notebook. These don't have to be organized, but make sure you use enough comments to understand the purpose of each code cell. Then, after you're done with your analysis, create a duplicate notebook where you will trim the excess and organize your steps so that you have a flowing, cohesive report.

In [ ]:
# Load your data and print out a few lines. Perform operations to inspect data
#   types and look for instances of missing or possibly errant data.



### Data Cleaning
> **Tip**: Make sure that you keep your reader informed on the steps that you are taking in your investigation. Follow every code cell, or every set of related code cells, with a markdown cell to describe to the reader what was found in the preceding cell(s). Try to make it so that the reader can then understand what they will be seeing in the following cell(s).
 

In [ ]:
# After discussing the structure of the data and any problems that need to be
#   cleaned, perform those cleaning steps in the second part of this section.


<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. **Compute statistics** and **create visualizations** with the goal of addressing the research questions that you posed in the Introduction section. You should compute the relevant statistics throughout the analysis when an inference is made about the data. Note that at least two or more kinds of plots should be created as part of the exploration, and you must  compare and show trends in the varied visualizations. 



> **Tip**: - Investigate the stated question(s) from multiple angles. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables. You should explore at least three variables in relation to the primary question. This can be an exploratory relationship between three variables of interest, or looking at how two independent variables relate to a single dependent variable of interest. Lastly, you  should perform both single-variable (1d) and multiple-variable (2d) explorations.


### Research Question 1 (Replace this header name!)

In [ ]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.


### Research Question 2  (Replace this header name!)

In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed in relation to the question(s) provided at the beginning of the analysis. Summarize the results accurately, and point out where additional research can be done or where additional information could be useful.

> **Tip**: Make sure that you are clear with regards to the limitations of your exploration. You should have at least 1 limitation explained clearly. 

> **Tip**: If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work here, check over your report to make sure that it is satisfies all the areas of the rubric (found on the project submission page at the end of the lesson). You should also probably remove all of the "Tips" like this one so that the presentation is as polished as possible.

## Submitting your Project 

> **Tip**: Before you submit your project, you need to create a .html or .pdf version of this notebook in the workspace here. To do that, run the code cell below. If it worked correctly, you should get a return code of 0, and you should see the generated .html file in the workspace directory (click on the orange Jupyter icon in the upper left).

> **Tip**: Alternatively, you can download this report as .html via the **File** > **Download as** submenu, and then manually upload it into the workspace directory by clicking on the orange Jupyter icon in the upper left, then using the Upload button.

> **Tip**: Once you've done this, you can submit your project by clicking on the "Submit Project" button in the lower right here. This will create and submit a zip file with this .ipynb doc and the .html or .pdf version you created. Congratulations!

In [ ]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Investigate_a_Dataset.ipynb'])